---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [6]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [9]:
def graph_identification():
    for i, G in enumerate(P1_Graphs, start=1):
        degrees = list(G.degree().values())
        mean_d = np.mean(degrees)
        min = np.min(degrees)
        max = np.max(degrees)
        mean_sp = nx.average_shortest_path_length(G)
        mean_cl = nx.average_clustering(G)
        print('G{} - d_mean: {:.2f}, d_min: {}, d_max: {}, '
              'mean_sp: {:.2f}, mean_cl: {:.2f}'
              .format(i, mean_d, min, max, mean_sp, mean_cl))
    return ['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

In [10]:
graph_identification()

G1 - d_mean: 3.99, d_min: 2, d_max: 100, mean_sp: 4.10, mean_cl: 0.03
G2 - d_mean: 10.00, d_min: 7, d_max: 13, mean_sp: 5.09, mean_cl: 0.56
G3 - d_mean: 4.00, d_min: 2, d_max: 6, mean_sp: 9.38, mean_cl: 0.40
G4 - d_mean: 7.96, d_min: 4, d_max: 92, mean_sp: 3.10, mean_cl: 0.04
G5 - d_mean: 4.00, d_min: 2, d_max: 11, mean_sp: 5.08, mean_cl: 0.00


['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [13]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [16]:
def salary_predictions():
    from sklearn.svm import SVC
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.metrics import roc_auc_score

    df = pd.DataFrame(index=G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['degree'] = pd.Series(G.degree())
    df['clust'] = pd.Series(nx.clustering(G))
    df['deg_c'] = pd.Series(nx.degree_centrality(G))
    df['close_c'] = pd.Series(nx.closeness_centrality(G))
    df['btwn_c'] = pd.Series(nx.betweenness_centrality(G))
    df['pg_rank'] = pd.Series(nx.pagerank(G, alpha=0.85))
    df['hits_hub'] = pd.Series(nx.hits(G)[0])
    df['hits_aut'] = pd.Series(nx.hits(G)[1])
    df['m_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))

    keep = df[~df['m_salary'].isnull()]
    hold = df[df['m_salary'].isnull()]
    X_keep = keep.drop('m_salary', axis=1)
    y_keep = keep['m_salary']
    X_hold = hold.drop('m_salary', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X_keep, y_keep,
                                                        random_state=1337)
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)
    X_hold_scaled = scaler.fit_transform(X_hold)

    param_grid = {'C':[10 ** a for a in range(-6,2)],
                  'gamma':[10 ** a for a in range(-6, -2)],
                  'class_weight':[None, 'balanced'],
                  'kernel':['rbf', 'linear']}
    svc = SVC(probability=True, random_state=0)
    clf = GridSearchCV(svc, param_grid, scoring='roc_auc')
    clf.fit(X_train_scaled, y_train)
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test_scaled)[:, 1])
    answer = pd.Series(data=clf.predict_proba(X_hold_scaled)[:, 1],
                       index=X_hold.index)
    return answer

In [17]:
salary_predictions()

1       0.116203
2       0.719094
5       0.996943
8       0.161313
14      0.277547
18      0.353689
27      0.320732
30      0.475820
31      0.186461
34      0.136925
37      0.059934
40      0.140928
45      0.187989
54      0.307596
55      0.440728
60      0.113098
62      0.999984
65      0.752151
77      0.102295
79      0.204485
97      0.087177
101     0.039768
103     0.284884
108     0.150807
113     0.585231
122     0.018100
141     0.441508
142     0.917534
144     0.075182
145     0.673723
          ...   
913     0.019613
914     0.060505
915     0.000053
918     0.163121
923     0.007181
926     0.074533
931     0.034266
934     0.002896
939     0.001657
944     0.000075
945     0.013346
947     0.169215
950     0.117606
951     0.010732
953     0.009913
959     0.000173
962     0.000075
963     0.182191
968     0.068929
969     0.072667
974     0.104398
984     0.002315
987     0.108305
989     0.079079
991     0.083213
992     0.000361
994     0.000191
996     0.0000

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [19]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [24]:
def new_connections_predictions():
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    def communities(row):
        """
        Check to whether are in the same department or notself.
        Vectorized for rows, use with pd.DataFrame.apply(x, axis=1)
        """
        nodes = row.name
        a = nodes[0]
        b = nodes[1]
        comm_a = G.node[a]['Department']
        comm_b = G.node[b]['Department']
        if comm_a == comm_b:
            return 1
        else:
            return 0
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    future_connections['same_comm'] = future_connections.apply(communities,
                                                               axis=1)
    # For Soundarajan-Hopcroft algorithms.
    for node in G.nodes():
       G.node[node]['community'] = G.node[node]['Department']

    pa = list(nx.preferential_attachment(G))
    pa_df = pd.DataFrame(index=[(i[0], i[1]) for i in pa],
                         data={'pref_att':[i[2] for i in pa]})

    cn = [(e[0], e[1], len(list(nx.common_neighbors(G, e[0], e[1]))))
          for e in nx.non_edges(G)]
    cn_df = pd.DataFrame(index=[(i[0], i[1]) for i in cn],
                         data={'comm_neigh':[i[2] for i in cn]})

    cnsh = list(nx.cn_soundarajan_hopcroft(G))
    cnsh_df = pd.DataFrame(index=[(i[0], i[1]) for i in cnsh],
                         data={'sh_comm_neigh':[i[2] for i in cnsh]})

    ra = list(nx.resource_allocation_index(G))
    ra_df = pd.DataFrame(index=[(i[0], i[1]) for i in ra],
                         data={'reso_alloc':[i[2] for i in ra]})

    rash = list(nx.ra_index_soundarajan_hopcroft(G))
    rash_df = pd.DataFrame(index=[(i[0], i[1]) for i in rash],
                         data={'sh_reso_alloc':[i[2] for i in rash]})

    jc = [i for i in nx.jaccard_coefficient(G)]
    jc_df = pd.DataFrame(index=[(i[0], i[1]) for i in jc],
                         data={'jacc_coeff':[i[2] for i in jc]})

    for df in [pa_df, cn_df, cnsh_df, ra_df, rash_df, jc_df]:
        future_connections = future_connections.merge(df, how='left',
                                                      left_index=True,
                                                      right_index=True)

    keep = future_connections[~future_connections['Future Connection'].isnull()]
    hold = future_connections[future_connections['Future Connection'].isnull()]

    X_keep = keep.drop('Future Connection', axis=1)
    y_keep = keep['Future Connection']
    X_hold = hold.drop('Future Connection', axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X_keep, y_keep,
                                                        random_state=0)
    clf = LogisticRegression(random_state=0)
    clf.fit(X_train, y_train)

    # Check on ROC_AUC performance.
    roc_auc = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

    probs = clf.predict_proba(X_hold)[:, 1]
    answer = pd.Series(index=X_hold.index,
                       data=probs)
    return answer

In [25]:
new_connections_predictions()

(107, 348)    0.028678
(542, 751)    0.010855
(20, 426)     0.590880
(50, 989)     0.010942
(942, 986)    0.011036
(324, 857)    0.010930
(13, 710)     0.157245
(19, 271)     0.128695
(319, 878)    0.010972
(659, 707)    0.010864
(49, 843)     0.011000
(208, 893)    0.010885
(377, 469)    0.007668
(405, 999)    0.018719
(129, 740)    0.017236
(292, 618)    0.018363
(239, 689)    0.010978
(359, 373)    0.008900
(53, 523)     0.328596
(276, 984)    0.010995
(202, 997)    0.011013
(604, 619)    0.036774
(270, 911)    0.010977
(261, 481)    0.068171
(200, 450)    0.999803
(213, 634)    0.010828
(644, 735)    0.035077
(346, 553)    0.010612
(521, 738)    0.010201
(422, 953)    0.017386
                ...   
(672, 848)    0.010977
(28, 127)     0.941715
(202, 661)    0.010575
(54, 195)     0.999723
(295, 864)    0.010916
(814, 936)    0.010841
(839, 874)    0.011036
(139, 843)    0.010924
(461, 544)    0.009918
(68, 487)     0.009869
(622, 932)    0.010895
(504, 936)    0.016175
(479, 528) 